In [1]:
import re,os
import numpy as np
import random
mypath="authors"

In [2]:
def dataSet(dirName):
    authors=[]
    path=os.getcwd()+'/'+dirName+'/'
    mylist=[]
    for filename in os.listdir(path):
        authors.append(filename)
        for file in os.listdir(path+filename):
            files=open(path+filename+'/'+file,'r',encoding="utf-8")
            text=files.read().split()
            mylist.append(text) 
    return mylist,authors

In [3]:
dataset,authors=dataSet(mypath)

dataset=[[w.lower() for w in line] for line in dataset] #lowercase


In [4]:
docData=[]
docAuthors=[]
defaultDocWord=530
for i in range(len(authors)):
    lenBook=len(dataset[i])
    lenDoc=int(lenBook/defaultDocWord)
    if(lenBook%defaultDocWord!=0):
        lenDoc=lenDoc+1
    for k in range(lenDoc):
        docData.append(dataset[i][k*defaultDocWord:(k+1)*defaultDocWord])
        docAuthors.append(authors[i]) 


In [5]:

def divideDataSet(docData,docAuthors):
   
    suffl = list(zip(docData, docAuthors))
    random.shuffle(suffl)
    docData, docAuthors = zip(*suffl)

    lenTrainSet=int(len(docData)*0.8)
    
    trainSetData=docData[0:lenTrainSet]
    trainSetAuthors=docAuthors[0:lenTrainSet]
    
    testSetData=docData[lenTrainSet:len(docData)]
    testSetAuthors=docAuthors[lenTrainSet:len(docAuthors)]
    return trainSetData,trainSetAuthors,testSetData,testSetAuthors


In [6]:
trainSetData,trainSetAuthors,testSetData,testSetAuthors = divideDataSet(docData,docAuthors)

In [7]:
from collections import Counter
def counter(docData):
    wordcount = Counter(docData)
    return wordcount

In [8]:
def joinWord():
    authSet=[]
    for auth in authors:
        index=[]
        wordSet=[]
        index = [index for index, value in enumerate(trainSetAuthors) if value == auth]
        for i in index:
            wordSet.extend(trainSetData[i])
        authSet.append(wordSet)
    return authSet

In [9]:
#Distinct Word joining
authSet=joinWord() # Kelimelerin sınıflandırılmış listesi
auth_word=[counter(authSet[i]) for i in range(len(authors))] #Distinct kelimeler

In [23]:
def match_list(docData):
    len_auth=len(authors)
    matching_list=[]
    for word in docData:
        tmp_list=[]
        for i in range(len_auth):
                tmp_list.append(auth_word[i][word])
        matching_list.append(tmp_list)
    return matching_list
    

In [25]:
tmp_list=match_list(testSetData[0])
def smoothing(tmp_list):
    for i in range(len(authors)):
        for k in range(len(tmp_list)):
            tmp_list[k][i]=(tmp_list[k][i]+1)/(len(authSet[i])+len(auth_word[i]))
    return tmp_list


In [79]:

from math import log

def prob(matrix):
    
    prob_list=np.zeros(len(authors))
    lenAuthors=len(authors)
    lenMatrix=len(matrix)
    for k in range(lenMatrix):
        for i in range(lenAuthors):
            sum_row=sum(matrix[k])
            prob_list[i]=prob_list[i]+log(matrix[k][i]/sum_row)
            
    
    return prob_list
    
    

In [85]:
prob_list=[]
for i in range(len(testSetData)):
    matrix=np.array(smoothing(match_list(testSetData[i])))
    max_ind=np.argmax(prob(matrix))
    prob_list.append(prob(matrix))
    print("->" + authors[max_ind] + " : " + testSetAuthors[i])



->Oguz Atay : Oguz Atay
->Andres platonov : Andres platonov
->Maksim Gorki : James Joyce
->Maksim Gorki : James Joyce
->Oguz Atay : Oguz Atay
->Andres platonov : Andres platonov
->Andres platonov : Andres platonov
->Ahmet hamdi : Ahmet hamdi
->Yasar Kemal : Yasar Kemal
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Oguz Atay : Oguz Atay
->Maksim Gorki : Maksim Gorki
->Maksim Gorki : James Joyce
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->James Joyce : James Joyce
->Maksim Gorki : James Joyce
->Oguz Atay : Oguz Atay
->Yasar Kemal : Yasar Kemal
->Maksim Gorki : James Joyce
->Andres platonov : Andres platonov
->Maksim Gorki : James Joyce
->Maksim Gorki : Maksim Gorki
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Maksim Gorki : James Joyce
->Ahmet hamdi : Ahmet hamdi
->Yasar Kemal : Yasar Kemal
->Yasar Kemal : Yasar Kemal
->Maksim Gorki : Maksim Gorki
->Yasar Kemal : Yasar Kemal
->Oguz Atay : Oguz Atay
->Yasar Kemal : Yasar Kemal
->Andres platonov : Andres platonov
->And

->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Oguz Atay : Oguz Atay
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Maksim Gorki : James Joyce
->James Joyce : James Joyce
->Ahmet hamdi : James Joyce
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Andres platonov : Andres platonov
->Yasar Kemal : Yasar Kemal
->Ahmet hamdi : Ahmet hamdi
->Yasar Kemal : Yasar Kemal
->Andres platonov : Andres platonov
->Ahmet hamdi : Oguz Atay
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Yasar Kemal : Yasar Kemal
->Maksim Gorki : James Joyce
->Yasar Kemal : Yasar Kemal
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Yasar Kemal : Yasar Kemal
->Ahmet hamdi : Ahmet hamdi
->Maksim Gorki : James Joyce
->Yasar Kemal : Yasar Kemal
->Oguz Atay : Oguz Atay
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Gabriel Garcia Marquez : Gabriel Garcia Marquez
->Oguz Atay : Oguz Atay
->James Joyce : James Joyce
->Ahmet hamdi : Ahmet hamdi
->Maksim Gorki : James Joyce
->Ahmet hamdi : James Joyce


In [87]:
def conf_matrix(cos_list):
    conf_mat=np.zeros((len(authors),len(authors)))
    for i in range(len(cos_list)):
        max_in=np.argmax(cos_list[i])
        #print(authors[max_in]," : ", testSetAuthors[i])
        conf_mat[authors.index(testSetAuthors[i])][max_in]=conf_mat[authors.index(testSetAuthors[i])][max_in]+1

    return conf_mat

In [98]:
matrix=conf_matrix(prob_list)

In [99]:
def  pre_rec(matrix,i):

    colm_pred=sum(matrix[:,i])
    row_pred=sum(matrix[i])
    prec=matrix[i][i]/colm_pred
    recall=matrix[i][i]/row_pred

    return prec,recall

In [100]:
def f1_score():
    prec_average=0
    recall_average=0
    lenAuthors=len(authors)
    for i in range(lenAuthors):
        prec,recall=pre_rec(matrix,i)
        prec_average=prec_average+prec/lenAuthors
        recall_average=recall_average+recall/lenAuthors
    f1_score=2*(prec_average*recall_average)/(prec_average+recall_average)
    return f1_score

In [101]:
f1_score()

0.86639612144597544

In [102]:
def accuracy(matrix):
  
    all_pred=sum(sum(matrix))
    t_pred=sum(np.diagonal(matrix))
    return (t_pred/all_pred)

In [113]:
accuracy(matrix)
matrix

array([[ 39.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  32.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,  38.,   0.,   0.,   0.,   0.],
       [  1.,  43.,   0.,  33.,   0.,   0.,  13.],
       [  0.,   0.,   0.,   0.,  58.,   0.,   1.],
       [  0.,   5.,   0.,   0.,   0.,  52.,   8.],
       [  0.,   0.,   0.,   0.,   0.,   0.,  34.]])